# Notebook 10: A hyperparameter search

### Goal: Training a ML using all features/predictors/inputs and all ML methods

#### Reminder of Problem Statement

Reminder of the ML task we want to accomplish in the paper. 

1. Does this image contain a thunderstorm? <-- Classification
2. How many lightning flashes are in this image? <-- Regression

#### Background

Please make sure you already did Notebook 5, because this notebook extends what we did in Notebook 5 to now include many more input predictors.

#### Step 1 & 2: Import packages and load data for Classification  
In Notebook 5 we only wanted 1 feature, no we want all available inputs (36 total). So all we need to change is the ```features_to_keep``` keyword to inlcude all indices. 